# Ejemplo para convertir un modelo a Torchscript
---

## 1. Instalar e importar las librerías necesarias

En este ejemplo vamos a trabajar con Pytorch y los modelos de torchvision

In [1]:
!pip3 install torch torchvision torchinfo numpy

In [2]:
from torchvision.models import resnet18, ResNet18_Weights, resnet50, ResNet50_Weights
from torchinfo import summary
import torch
import torchvision
import time
import numpy as np

## 2. Definir el modelo

Definimos el modelo, en este caso, usamos ResNet50 pre-entrenada en ImageNet.

In [4]:
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights)
preprocessing = ResNet50_Weights.IMAGENET1K_V1.transforms()
summary(model, input_size=(1, 3, 224, 224))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 78.1MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

## 6. Exportar el modelo en INT8

Una vez que hemos realizado el entrenamiento para pasar a INT8, simplemente limpiamos las capas auxiliares que añade Pytorch para realizar la calibración y exportamos el modelo a TorchScript para poder usarlo en un móvil.

In [5]:
model.eval().cpu()
model = torch.quantization.convert(model, inplace=True).cpu()
model_script = torch.jit.script(model) # Export to TorchScript
torch.jit.save(model_script, './model.pt')